In [256]:
import os 
import pandas as pd
import numpy as np
pd.options.display.max_rows = 3000

pathTesis = os.getcwd()
pathBases = os.path.join(pathTesis,'bases/DatosPrecia/PreciosConColumnas')
pathYield = os.path.join(pathTesis,'bases/DatosPrecia/yield')
pathMaturity = os.path.join(pathTesis,'bases/DatosPrecia/tablaMaturity')
pathSpline = os.path.join(pathTesis,'bases/DatosPrecia/tablaSpline')
pathibr = os.path.join(pathTesis,'bases/IBR')
pathndata = os.path.join(pathTesis,'bases/ndatos')

In [3]:
#List of Available bonds for each year
resumenMaturity = pd.read_csv('E:/Documentos/Python/TesisJuanFelipe/bases/ResumenMaturityN.csv',sep=";")

In [173]:
resumenMaturity.head()

,Nemotecnico,YtoM,Ano
0,TFIT02150114,1.0,2013
1,TFIT06140514,1.4,2013
2,TFIT10120914,1.7,2013
3,TFIT02121114,1.9,2013
4,TFIT10281015,2.8,2013


## Tasas Medias y Largas por año

In [175]:
medioLargo = []
for nom in os.listdir(pathYield):
    periodo = int(nom.replace('_yield.csv','')[-8:-4]) #year
    columnasIniciales = resumenMaturity[resumenMaturity['Ano']==periodo]['Nemotecnico'].values #bonds from the year
    print(periodo)
    dataSecundaria = pd.read_csv(os.path.join(pathYield,nom),sep='|')[['Fecha']+list(columnasIniciales)]
    dataSecundaria.columns = ['Fecha'] + list(resumenMaturity[resumenMaturity['Ano']==periodo]['YtoM'].values)
    dataSecundaria.index=dataSecundaria.Fecha
    medioLargo.append(dataSecundaria.drop(['Fecha'],axis=1))    

2013
2014
2015
2016
2017
2018


In [182]:
medioLargo[0].info()

<class 'pandas.core.frame.DataFrame'>
Index: 365 entries, 2013-01-01 to 2013-12-31
Data columns (total 14 columns):
1.0     363 non-null float64
1.4     365 non-null float64
1.7     364 non-null float64
1.9     363 non-null float64
2.8     365 non-null float64
2.9     364 non-null float64
3.5     365 non-null float64
5.9     329 non-null float64
5.8     365 non-null float64
7.6     365 non-null float64
9.3     365 non-null float64
11.6    365 non-null float64
13.7    365 non-null float64
15.3    356 non-null float64
dtypes: float64(14)
memory usage: 42.8+ KB


## Tasas Cortas IBR

In [109]:
def mktasas(x):
    try:
        return float(x[0:5])
    except ValueError:
        return np.nan

In [204]:
cortos = []
for nom in os.listdir(pathibr):
    dataCortos = pd.read_csv(os.path.join(pathibr,nom),sep=',',parse_dates=[0])
    dataCortos=dataCortos[(dataCortos['agente']=='IBR')&(dataCortos['tipo_tasa']=='Tasa nominal')]
    dataCortos['tasas']=dataCortos.valor_tasa.apply(mktasas)
    dataCortos=dataCortos[['fecha','tasas']]
    dataCortos.columns=['fecha',nom.replace('.csv','')]
    cortos.append(dataCortos)

In [206]:
ibr_o1=cortos[0].merge(cortos[3],on='fecha')
ibr_o13=ibr_o1.merge(cortos[2],on='fecha')
ibr=ibr_o13.merge(cortos[1],how='left',on='fecha')
ibr.columns=['Fecha','O','0.1','0.3','0.5']
ibr.index=ibr.Fecha
ibr=ibr.drop(['Fecha'],axis=1)

In [207]:
ibr.info(200)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1612 entries, 2012-01-08 to 2019-03-13
Data columns (total 4 columns):
O      1612 non-null float64
0.1    1612 non-null float64
0.3    1612 non-null float64
0.5    685 non-null float64
dtypes: float64(4)
memory usage: 63.0 KB


## Datos para cada año

In [248]:
yearid={'2013':0,'2014':1,'2015':2,'2016':3,'2017':4,'2018':5}

In [249]:
year='2013'

In [251]:
term=ibr[year].merge(medioLargo[yearid[year]],how='right',left_index=True,right_index=True)
term=term.fillna(method='pad', axis=0)

In [262]:
term.head()

,O,0.1,0.3,0.5,1.0,1.4,1.7,1.9,2.8,2.9,3.5,5.9,5.8,7.6,9.3,11.6,13.7,15.3
Fecha,,,,,,,,,,,,,,,,,,
2013-01-01,NaN,NaN,NaN,NaN,4.55,4.68,4.87,4.72,4.88,5.21,4.99,NaN,5.32,5.44,5.55,5.84,5.93,NaN
2013-01-02,3.838,3.843,3.770,NaN,4.55,4.69,4.87,4.73,4.91,5.21,4.99,NaN,5.31,5.46,5.53,5.85,5.89,NaN
2013-01-03,3.625,3.597,3.505,NaN,4.54,4.64,4.83,4.71,4.84,5.21,4.96,NaN,5.29,5.49,5.55,5.83,5.89,NaN
2013-01-04,3.090,3.125,3.128,NaN,4.55,4.60,4.82,4.71,4.85,5.21,4.97,NaN,5.30,5.55,5.57,5.85,5.90,NaN
2013-01-05,3.090,3.125,3.128,NaN,4.55,4.60,4.82,4.71,4.85,5.21,4.97,NaN,5.30,5.55,5.57,5.85,5.90,NaN


In [255]:
outdata=term.reset_index()
outdata.to_csv(os.path.join(pathndata,str(year)+'data.csv'),sep='|',index=False)